In [1]:
import tensorflow as tf

In [4]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

import gensim

import scikitplot as skplt

import nltk

import os

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Bidirectional
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.optimizers import Adam

Using TensorFlow backend.


### Model Training and Selection

In [ ]:
# need pandas dataframe sample_train

def evaluate_features(X, y, clf=None):

    if clf is None:
        clf = LogisticRegression()
    
    probas = cross_val_predict(clf, X, y, cv=StratifiedKFold(random_state=8), 
                              n_jobs=-1, method='predict_proba', verbose=2)
    pred_indices = np.argmax(probas, axis=1)
    classes = np.unique(y)
    preds = classes[pred_indices]
    print('Log loss: {}'.format(log_loss(y, probas)))
    print('Accuracy: {}'.format(accuracy_score(y, preds)))
    skplt.metrics.plot_confusion_matrix(y, preds, figsize=(12,8))
    skplt.metrics.plot_precision_recall_curve(y, probas, figsize=(12,8))
    plt.show()


In [ ]:
# nn_model = tf.keras.models.Sequential([
#   tf.keras.layers.Dense(?, activation='relu'),
#   tf.keras.layers.Dense(?, activation='relu'),
#   tf.keras.layers.Dropout(0.2),
#   tf.keras.layers.Dense(?number of types, activation='sigmoid')
# ])

In [ ]:
# Use the Keras tokenizer
embed_dim = ?
lstm_out = ?

num_words = 2000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(sample_train['Text'].values)

# Pad the data 
X = tokenizer.texts_to_sequences(sample_train['Text'].values)
X = pad_sequences(X, maxlen=2000)

? output activation use softmax or sigmoid

lstm_model = tf.keras.models.Sequential([
  tf.keras.layers.Embedding(num_words, embed_dim),
  tf.keras.layers.LSTM(lstm_out, recurrent_dropout=0.2, dropout=0.2),
  tf.keras.layers.Dense(?,activation='sigmoid/softmax'),
])

In [ ]:
embed_dim = ?
lstm_out = ?

num_words = 2000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(sample_train['Text'].values)

# Pad the data 
X = tokenizer.texts_to_sequences(sample_train['Text'].values)
X = pad_sequences(X, maxlen=2000)

X_val = tokenizer.texts_to_sequences(sample_test['Text'].values)
X_val = pad_sequences(X_val, maxlen=2000)

? output activation use softmax or sigmoid

bidirectional_lstm_model = tf.keras.models.Sequential([
  tf.keras.layers.Embedding(num_words, embed_dim),
  tf.keras.layers.Bidirectional(LSTM(64)),
  tf.keras.layers.Dropout(0.5)
  tf.keras.layers.Dense(?,activation='sigmoid/softmax'),
])

tf-idf word embeddings

In [ ]:
# need pandas dataframe sample_train

# count_vectorizer = TfidfVectorizer(
#     analyzer="word", tokenizer=nltk.word_tokenize,
#     preprocessor=None, stop_words='english', max_features=None)    

# tfidf = count_vectorizer.fit_transform(sample_train['text'])
# tfidf_test = count_vectorizer.fit_transform(sample_test['text'])

# len(count_vectorizer.get_feature_names())

In [ ]:
# svd = TruncatedSVD(n_components=25, n_iter=25, random_state=12)
# truncated_tfidf = svd.fit_transform(tfidf)
# truncated_tfidf_test = svd.fit_transform(tfidf_test)

In [ ]:
# compile the keras model
# nn_model

# X_train = truncated_tfidf
# y_train = sample_train['type']

# X_test = truncated_tfidf_test
# y_test = sample_test['type']

# nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# nn_model.fit(X_train, y_train, epochs=300, batch_size=20, verbose=0)

# _,train_accuracy = nn_model.evaluate(X_train, y_train, verbose=0)
# print('Training Accuracy: %.2f' % (train_accuracy*100))

# _,test_accuracy = nn_model.evaluate(X_test, y_test, verbose=0)
# print('Test Accuracy: %.2f' % (test_accuracy*100))

In [ ]:
# evaluate_features(truncated_tfidf, sample_train['type'].values.ravel(), clf = nn_model)

In [ ]:
# compile the keras model
# lstm_model

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(X, y_train, epochs=8, batch_size=20, validation_split=0.2, verbose=0)

_,train_accuracy = lstm_model.evaluate(X, y_train, verbose=0)
print('Training Accuracy: %.2f' % (train_accuracy*100))

_,test_accuracy = lstm_model.evaluate(X_val, y_test, verbose=0)
print('Test Accuracy: %.2f' % (test_accuracy*100))

In [ ]:
# compile the keras model
# bidirectional_lstm_model

bidirectional_lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
bidirectional_lstm_model.fit(X, y_train, epochs=8, batch_size=20, validation_split=0.2, verbose=0)

_,train_accuracy = bidirectional_lstm_model.evaluate(X, y_train, verbose=0)
print('Training Accuracy: %.2f' % (train_accuracy*100))

_,test_accuracy = bidirectional_lstm_model.evaluate(X_val, y_test, verbose=0)
print('Test Accuracy: %.2f' % (test_accuracy*100))

word2vec word embeddings

In [ ]:
# class MySentences(object):

#     def __init__(self, *arrays):
#         self.arrays = arrays
 
#     def __iter__(self):
#         for array in self.arrays:
#             for document in array:
#                 for sent in nltk.sent_tokenize(document):
#                     yield nltk.word_tokenize(sent)

# def get_word2vec(sentences, location):

#     if os.path.exists(location):
#         print('Found {}'.format(location))
#         model = gensim.models.Word2Vec.load(location)
#         return model
    
#     print('{} not found. training model'.format(location))
#     model = gensim.models.Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)
#     print('Model done training. Saving to disk')
#     model.save(location)
#     return model

In [ ]:
# w2vec = get_word2vec(
#     MySentences(
#         sample_train['text'].values, 
#     ),
#     'w2vmodel'
# )
# w2vec_test = get_word2vec(
#     MySentences(
#         sample_test['text'].values, 
#     ),
#     'w2vmodel_test'
# )

In [ ]:
# class MyTokenizer:
#     def __init__(self):
#         pass
    
#     def fit(self, X, y=None):
#         return self
    
#     def transform(self, X):
#         transformed_X = []
#         for document in X:
#             tokenized_doc = []
#             for sent in nltk.sent_tokenize(document):
#                 tokenized_doc += nltk.word_tokenize(sent)
#             transformed_X.append(np.array(tokenized_doc))
#         return np.array(transformed_X)
    
#     def fit_transform(self, X, y=None):
#         return self.transform(X)

# class MeanEmbeddingVectorizer(object):
#     def __init__(self, word2vec):
#         self.word2vec = word2vec
#         self.dim = len(word2vec.wv.syn0[0])

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         X = MyTokenizer().fit_transform(X)
        
#         return np.array([
#             np.mean([self.word2vec.wv[w] for w in words if w in self.word2vec.wv]
#                     or [np.zeros(self.dim)], axis=0)
#             for words in X
#         ])
    
#     def fit_transform(self, X, y=None):
#         return self.transform(X)

In [ ]:
# mean_embedding_vectorizer = MeanEmbeddingVectorizer(w2vec)
# mean_embedding_vectorizer_test = MeanEmbeddingVectorizer(w2vec_test)

# mean_embedded = mean_embedding_vectorizer.fit_transform(sample_train['text'])
# mean_embedded_test = mean_embedding_vectorizer_test.fit_transform(sample_test['text'])

In [ ]:
# compile the keras model
# nn_model

# X_train = mean_embedded
# y_train = sample_train['type']

# X_test = mean_embedded_test
# y_test = sample_test['type']

# nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# nn_model.fit(X_train, y_train, epochs=300, batch_size=20, verbose=0)

# _,train_accuracy = nn_model.evaluate(X_train, y_train, verbose=0)
# print('Training Accuracy: %.2f' % (train_accuracy*100))

# _,test_accuracy = nn_model.evaluate(X_test, y_test, verbose=0)
# print('Test Accuracy: %.2f' % (test_accuracy*100))

In [ ]:
# evaluate_features(mean_embedded, sample_train['type'].values.ravel(), clf = nn_model)

### First model

In [ ]:
# need selected word embedding and model

selected_model = ?

# probas = cross_val_predict(selected_model, example_X, example_y, cv=StratifiedKFold(random_state=8), 
#                               n_jobs=-1, method='predict_proba', verbose=2)

probas = selected_model.predict_proba(example_X)
pred_indices = np.argmax(probas, axis=1)

# here classes needed
preds = classes[pred_indices]

Entity Linking and Typing

In [ ]:
def get_entities(word):
    return

def get_number_links(entity, type):
    return

def prob_entity_link_to_type(entities_of_word, word_type):
    links = []
    for entity in entities_of_word:
        links.append(get_number_links(entity, word_type))
    links = np.array(links)
    return links/np.sum(links)

In [ ]:
entity_sentence = []

for idx, word in enumerate(sentence):
    word_type = 
    entities_of_word = get_entities(word)
    prob = probas[pred_indices[idx]]*prob_entity_link_to_type(entities_of_word, word_type)
    entity_sentence.append(entities_of_word(prob.index(max(prob))))

### Second Model

In [ ]:
additional_model = ?

new_sentence = []
for idx, word in enumerate(sentence):
    word_type = preds[idx]
    new_sentence.append(word_type)

def predict_entity(sentence):
    entities = []
    for idx in range(len(sentence)):
        word = sentence[idx]
        new_sentence = []
        for i in range(len(sentence)):
            if i != idx:
                word_type = preds[idx]
                new_sentence.append(word_type)
            else:
                new_sentence.append(sentence[idx])
        new_probas = selected_model.predict_proba(new_sentence)
        new_prob = new_probas[idx]
        new_prob_idx = new_prob.index(max(new_prob))
        
        word_type = preds[new_prob_idx]
        entities_of_word = get_entities(word)
        prob = new_prob[new_prob_idx]*prob_entity_link_to_type(entities_of_word, word_type)
        entities.append(entities_of_word(prob.index(max(prob))))
    return entities